In [1]:
import requests
import json
import re

from collections import Counter

from homeworks.utils.aidevsApi import get_auth_token, get_task, send_answer, API_KEY_OPENAI
from openai import OpenAI

client = OpenAI(api_key=API_KEY_OPENAI)

token = get_auth_token('google')
task = get_task(token, print_task=True)

----------- Task description -----------
msg: Provide me the URL to your API (HTTPS) via /answer/ endpoint. I will ask your API a question that requires search engine integration. Your job is to provide me answer to my question
hint1: Please use SerpAPI or similar service. https://serpapi.com (free account is enough)
hint2: Probably I will ask more than one question, so be prepared for that
hint3: Please return the answer in JSON format, with "reply" field!
hint4: Return as concise an answer as possible
-----------    ----------    -----------


In [2]:
# question_url = "give me URL to polish portal called Onet"
question_url = """Portal niebezpiecznik.pl napisał kiedyś artykuł ma temat zastrzegania numeru pesel. 
Mozesz zwrocic mi adres URL do niego? Potrzebuje do tego najnowszego."""

In [3]:
# testing my temporary API:
url = "https://myownapiv3.azurewebsites.net/question_url/"

data = {
    "question": question_url
}
headers = {
    "Content-Type": "application/json"
}

data_json = json.dumps(data)

response = requests.post(url, data=data_json, headers=headers)

response.json()

{'reply': 'https://niebezpiecznik.pl/post/zastrzeganie-numeru-pesel-juz-od-jutra-tlumaczymy-jak-to-bedzie-dzialac/'}

In [4]:
token = get_auth_token('google')
answer = url
send_answer(token, answer)

Sending answer: done! :)


#### Additional code used for API :)

In [36]:
def google_search_question(question_url):
    # ask LLM compressing
    response = client.chat.completions.create(
        model="gpt-4-0613",
        messages=[
            {"role": "system", "content": "Transform given prompt into a google search query."},
            {"role": "user", "content": f"User prompt: ###{question_url}###"}
        ],
        )
    
    q = response.choices[0].message.content
    
    return q


def serpApiQuestion(prompt):
    # SerpApi endpoint
    url = "https://serpapi.com/search"
    
    # Your SerpApi private key
    api_key = "...give here your onw API key ;) ..."
    
    # Parameters for the search query
    params = {
        "q": prompt,
        "engine": "google",
        "device": "desktop",
        "api_key": api_key,
    }

    # Send the GET request to SerpApi
    response = requests.get(url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Print the JSON response
        return(response.json())
    else:
        # Print an error message if the request was not successful
        print(f"Error: {response.status_code}, {response.text}")            

In [37]:
google_prompt = google_search_question(question_url).replace('"','')
google_prompt

'Niebezpiecznik.pl artykuł o zastrzeganiu numeru pesel'

In [38]:
serp_answer = serpApiQuestion(google_prompt)

In [42]:
serp_answer['organic_results'][0]['link'] # pierwszy z brzegu

'https://niebezpiecznik.pl/post/zastrzeganie-numeru-pesel-juz-od-jutra-tlumaczymy-jak-to-bedzie-dzialac/'